In [ ]:
import json
import csv
import pandas as pd

# Read JSONL file
data = []
with open('sample_data/ont_1_university_train.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        data.append(json.loads(line))

# Convert to DataFrame and save as CSV
df = pd.DataFrame(data)
df.to_csv('sample_data/ont_1_university_train.csv', index=False)

In [ ]:
import tensorflow as tf
import pandas as pd

FILE = "ont_1_university_test"

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value.encode()]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# Read the CSV
df = pd.read_csv(f'sample_data/{FILE}.csv')

# Write to TFRecord
with tf.io.TFRecordWriter(f'sample_data/{FILE}.tfrecord') as writer:
    for _, row in df.iterrows():
        # Safely convert string representation of list to actual Python list
        # triples_list = ast.literal_eval(row['triples'])
        # Convert to JSON string
        # formatted_triples = json.dumps(triples_list)
        
        feature = {
            'id': _bytes_feature(row['id']),
            'sent': _bytes_feature(row['sent']),
           # 'triples': _bytes_feature(formatted_triples)
        }
        
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())

print("Done")

In [ ]:
# First, verify your TFRecord format is correct
import tensorflow as tf

# Read and verify the TFRecord
raw_dataset = tf.data.TFRecordDataset(f'sample_data/{FILE}.tfrecord')

# The feature description Flan-T5 typically expects
feature_description = {
    'id': tf.io.FixedLenFeature([], tf.string),
    'sent': tf.io.FixedLenFeature([], tf.string),
    #'triples': tf.io.FixedLenFeature([], tf.string),  # If you have this field
}

def _parse_function(example_proto):
    return tf.io.parse_single_example(example_proto, feature_description)

# Parse and check the data
parsed_dataset = raw_dataset.map(_parse_function)
for parsed_record in parsed_dataset.take(2):
    print(parsed_record)

In [ ]:
import tensorflow as tf
from google.cloud import aiplatform
from google.cloud import storage
import os

import tensorflow as tf
from transformers import T5Config, TFT5ForConditionalGeneration

import os
import vertexai

# 1. Set up paths and configurations
LOCAL_MODEL_DIR = os.path.abspath("local_model")
SAVED_MODEL_DIR = os.path.abspath("saved_model")
GCS_BUCKET = "ontologykg2"
GCS_MODEL_PATH = f"gs://{GCS_BUCKET}/models/flan-t5-xxl"
PROJECT_ID = "deft-return-439619-h9"
REGION = "us-west1"

# Set up paths using os.path.join for cross-platform compatibility
LOCAL_CHECKPOINT_DIR = os.path.abspath("local_checkpoint/checkpoint_1000001")  # Use absolute path
LOCAL_EXPORT_DIR = os.path.abspath("exported_model")  # Use absolute path

print(f"Local model dir: {LOCAL_MODEL_DIR}")
print(f"SavedModel dir: {SAVED_MODEL_DIR}")

print(f"local checkpoint dir:{LOCAL_CHECKPOINT_DIR}")

# 2. Download and save FLAN-T5-base in .h5 format
print("Downloading FLAN-T5-base...")
model = TFT5ForConditionalGeneration.from_pretrained(
    "google/flan-t5-base",
    from_pt=True  # Convert from PyTorch to TF
)

# Save as .h5
os.makedirs(LOCAL_MODEL_DIR, exist_ok=True)
model.save_pretrained(LOCAL_MODEL_DIR)
print(f"Model saved to {LOCAL_MODEL_DIR}")

# 3. Convert to SavedModel format
print("Converting to SavedModel format...")
os.makedirs(SAVED_MODEL_DIR, exist_ok=True)



In [ ]:
import torch
from models.flant5.FlanT5 import T5FineTuner, tokenize_dataset

CKPT_PATH = "models/flant5/epoch=3-step=2072-train_loss=0.35.ckpt"
#CKPT_PATH = "lightning_logs/version_34/epoch=3-step=2072-train_loss=0.35.ckpt"
#CKPT_PATH = "lightning_logs/version_30/final.ckpt"

checkpoint = torch.load(CKPT_PATH)
print(checkpoint.keys())

llm = T5FineTuner.load_from_checkpoint(CKPT_PATH)

llm.model.eval() # set model to evaluation mode
llm = llm.to("cpu") # use CPU since I don't have GPU
print("Done")

In [ ]:
model = llm.model

# 1. Convert PyTorch model to TensorFlow
def convert_pt_to_tf(llm):
    """Convert PyTorch T5 model to TensorFlow"""
    from transformers import TFT5ForConditionalGeneration
    
    # Create TF model with same config
    tf_model = TFT5ForConditionalGeneration.from_pretrained(
        llm.hparam.model_name_or_path,
        from_pt=True,
        config=llm.model.config
    )
    
    # Verify the conversion
    print("Model converted from PyTorch to TensorFlow")
    print(f"Model type: {type(tf_model)}")

    return tf_model

# Convert PyTorch model to TF
tf_model = convert_pt_to_tf(llm)
model = tf_model  # Use converted model for serving

In [ ]:
@tf.function(input_signature=[{
    'input_ids': tf.TensorSpec(shape=(1, 512), dtype=tf.int32, name='input_ids'),
    'attention_mask': tf.TensorSpec(shape=(1, 512), dtype=tf.int32, name='attention_mask')
}])
def serving_fn(inputs):
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=128,
        num_beams=4,
        pad_token_id=model.config.pad_token_id,
        eos_token_id=model.config.eos_token_id,
        bos_token_id=model.config.bos_token_id,
        use_cache=True,
        do_sample=False,
        num_return_sequences=1,
        return_dict_in_generate=True,
        output_scores=True
    )
    return {'sequences': outputs.sequences}


# 3. Test the serving function
def test_serving_fn():
    # Create dummy input
    dummy_input = {
        'input_ids': tf.zeros((1, 512), dtype=tf.int32),
        'attention_mask': tf.ones((1, 512), dtype=tf.int32)
    }
    
    # Test the function
    result = serving_fn(dummy_input)
    print("Serving function test successful")
    return result

# Test before saving
test_serving_fn()

In [ ]:
# 3. Save as SavedModel
tf.saved_model.save(
    model,
    SAVED_MODEL_DIR,
    signatures={
        'serving_default': serving_fn
    }
)
print(f"SavedModel saved to {SAVED_MODEL_DIR}")

In [ ]:
# 2. Create serving function
"""
@tf.function(input_signature=[{
    'input_ids': tf.TensorSpec(shape=(None, None), dtype=tf.int32, name='input_ids'),
    'attention_mask': tf.TensorSpec(shape=(None, None), dtype=tf.int32, name='attention_mask')
}])
def serving_fn(inputs):
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_length=128,
        num_beams=4,
        return_dict_in_generate=True,
        output_scores=True
    )
    return {'sequences': outputs.sequences}
"""
GCS_BUCKET = "ontologykg2"
GCS_MODEL_PATH = f"gs://{GCS_BUCKET}/model/flant5"

# 4. Upload to GCS and deploy to Vertex AI
print("Uploading to GCS and deploying to Vertex AI...")
vertexai.init(project=PROJECT_ID, location=REGION)

# Upload and create model
model = aiplatform.Model.upload(
    display_name="flan-t5-base",
    artifact_uri=GCS_MODEL_PATH,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-gpu.2-12:latest", 
)


In [ ]:

# Deploy to endpoint
endpoint = model.deploy(
    machine_type="n1-standard-4",  # Using GPU machine type for base
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,
    min_replica_count=1,
    max_replica_count=1,
)

print(f"Model deployed to endpoint: {endpoint.resource_name}")

In [ ]:
flant5 = FlanT5Client() #uses FLANT5_ENDPOINT set in utils.rag_constants.py by default
# Test prediction
test_text = "Translate to French: Hello, how are you?"
response = flant5.generate_content(test_text)
print(f"Test prediction response: {response}")

In [ ]:
GCS_EXPORT_DIR = ""
# Now upload the model to Vertex AI
model = aiplatform.Model.upload(
    display_name="flan-t5-custom",
    artifact_uri=GCS_EXPORT_DIR,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-12:latest",
)

In [ ]:
import vertexai
from vertexai.language_models import TextGenerationModel  # Changed import
from vertexai.generative_models import GenerativeModel

from app.utils.rag_constants import PROJECT_ID, REGION, GENERATIVE_MODEL

user_query = "What's a good name for a flower shop that specializes in selling bouquets of dried flowers?"

vertexai.init(project=PROJECT_ID, location=REGION)

MODEL_ID = "flant5-finetuned-model"
# Method 2: Using a model that you've fine-tuned
model = vertexai.Model(
    #name=f"projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}",
    name = endpoint
)
response = model.predict(instances=user_query)

"""
# Use pre-trained Flan-T5 directly
model = TextGenerationModel.from_pretrained("text-bison@001")  # or "text-bison@latest"
response = model.predict(user_query)

GENERATIVE_MODEL = "gemini-1.5-flash-002"
model = GenerativeModel(GENERATIVE_MODEL)

response = model.generate_content(
    "What's a good name for a flower shop that specializes in selling bouquets of dried flowers?"
)
"""

print(response.text)
# Example response:
# **Emphasizing the Dried Aspect:**
# * Everlasting Blooms
# * Dried & Delightful
# * The Petal Preserve
# ...